In [2]:
!git clone https://github.com/quannguyen2153/cam-combination.git

Cloning into 'cam-combination'...
remote: Enumerating objects: 1341, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 1341 (delta 72), reused 107 (delta 59), pack-reused 1220 (from 1)
Receiving objects: 100% (1341/1341), 240.34 MiB | 37.56 MiB/s, done.
Resolving deltas: 100% (767/767), done.


In [3]:
!pip install ttach

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import sys
import os
sys.path.append(os.path.abspath('/content/cam-combination'))

import json
import csv
import numpy as np
import cv2
import time
from multiprocessing.shared_memory import SharedMemory

import torch
import torchvision.models as models

from PIL import Image

import pytorch_grad_cam as cam
import pytorch_grad_cam.utils as utils
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget, ClassifierOutputSoftmaxTarget
from pytorch_grad_cam.utils.image import deprocess_image, preprocess_image

labels = json.load(open("/content/cam-combination/pytorch_grad_cam/utils/imagenet_class_index.json"))
img_names = json.load(open("/content/drive/My Drive/ThesisXAI/img_list.json"))["images"]
img_dir = "/content/drive/My Drive/ThesisXAI/evaluation_imgs"

def evaluate_conf(cam, model, target_layers, csv_file):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    shm = SharedMemory(name="activations", create=True, size=4)

    # Initialize metrics
    total_drop_in_conf = 0
    total_increase_in_conf = 0
    total_activations = 0
    total_time = 0
    num_images = 0  # Track number of processed images

    drop_in_conf_metric = utils.DropInConfidence()
    increase_in_conf_metric = utils.IncreaseInConfidence()

    # Header for the table
    print(f"| {'Progress':<10} | {'Image':<40} | {'Drop':<10} | {'Increase':<10} | {'Activations':<12} | {'Time':<10} |")

    # Open CSV file in append mode
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Write header row if the file is empty
        if file.tell() == 0:
            writer.writerow(["Progress", "Image", "Drop", "Increase", "Activations", "Time"])

        for idx, filename in enumerate(img_names):
            img_path = os.path.join(img_dir, filename)

            if os.path.exists(img_path):  # Ensure the file exists
                img = np.array(Image.open(img_path).convert("RGB"))
                img = cv2.resize(img, (224, 224))
                img = np.float32(img) / 255
                input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]).to(device)

                with torch.no_grad():
                    predicted_class = model(input_tensor).max(1)[-1]
                    targets = [ClassifierOutputTarget(predicted_class)]

                start_time = time.time()
                gray_scale = cam(model=model, target_layers=target_layers)(input_tensor=input_tensor, targets=targets)
                elapsed_time = time.time() - start_time
                num_activations = int.from_bytes(shm.buf[:4], byteorder='little')
                threshold = np.percentile(gray_scale, 50)
                gray_scale[gray_scale < threshold] = 0

                targets = [ClassifierOutputSoftmaxTarget(predicted_class)]

                # Calculate Drop in Confidence and Increase in Confidence
                drop_in_conf = drop_in_conf_metric(input_tensor, gray_scale, targets, model)
                increase_in_conf = increase_in_conf_metric(input_tensor, gray_scale, targets, model)

                # Accumulate the scores for averaging
                total_drop_in_conf += drop_in_conf
                total_increase_in_conf += increase_in_conf
                total_time += elapsed_time
                total_activations += num_activations

                num_images += 1  # Increment image count

                # Logging in table format
                print(f"| {idx}/{len(img_names):<8} | {filename:<40} | {drop_in_conf[0][0]:<10.5f} | {increase_in_conf[0][0]:<10.5f} | {num_activations:<12} | {elapsed_time:<10.5f} |")

                # Write to CSV file
                writer.writerow([f"{idx}/{len(img_names)}", filename, drop_in_conf[0][0], increase_in_conf[0][0], num_activations, elapsed_time])

    # Calculate and print averages after processing all images
    if num_images > 0:
        avg_drop_in_conf = total_drop_in_conf / num_images
        avg_increase_in_conf = total_increase_in_conf / num_images * 100
        avg_time = total_time / num_images
        avg_activations = total_activations / num_images
        print(f"| {'Average':<10} | {'-':<40} | {avg_drop_in_conf[0][0]:<10.5f} | {avg_increase_in_conf[0][0]:<10.5f} | {avg_activations:<12} | {avg_time:<10.5f} |")
    else:
        print("No valid images found in the specified folder.")

    shm.close()
    shm.unlink()

In [5]:
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT).eval().cuda()
target_layers = [model.features[29]]

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 84.0MB/s]


In [6]:
evaluate_conf(cam=cam.GradScoreCAM, model=model, target_layers=target_layers, csv_file="/content/drive/My Drive/ThesisXAI/grad_score_cam_evaluation.csv")

Using device: cuda
| Progress   | Image                                    | Drop       | Increase   | Activations  | Time       |


100%|██████████| 30/30 [00:01<00:00, 21.19it/s]


| 0/2000     | ILSVRC2012_val_00015720.JPEG             | 0.20791    | 0.00000    | 465          | 3.20211    |


100%|██████████| 29/29 [00:01<00:00, 21.92it/s]


| 1/2000     | ILSVRC2012_val_00015956.JPEG             | 78.80869   | 0.00000    | 460          | 1.98942    |


...

| 1997/2000     | ILSVRC2012_val_00020720.JPEG             | 1.79080    | 0.00000    | 466          | 2.16514    |


100%|██████████| 30/30 [00:01<00:00, 20.01it/s]


| 1998/2000     | ILSVRC2012_val_00008183.JPEG             | 74.33832   | 0.00000    | 478          | 2.22445    |


100%|██████████| 30/30 [00:01<00:00, 20.49it/s]


| 1999/2000     | ILSVRC2012_val_00031879.JPEG             | 0.00000    | 1.00000    | 470          | 2.19232    |
| Average    | -                                        | 41.51647   | 17.30000   | 466.278      | 2.16583    |


In [10]:
evaluate_conf(cam=cam.ScoreCAM, model=model, target_layers=target_layers, csv_file="/content/drive/My Drive/ThesisXAI/score_cam_evaluation.csv")

Using device: cuda
| Progress   | Image                                    | Drop       | Increase   | Activations  | Time       |


100%|██████████| 32/32 [00:02<00:00, 13.73it/s]


| 0/2000     | ILSVRC2012_val_00015720.JPEG             | 0.20791    | 0.00000    | 512          | 2.36927    |


100%|██████████| 32/32 [00:02<00:00, 14.07it/s]


| 1/2000     | ILSVRC2012_val_00015956.JPEG             | 78.85773   | 0.00000    | 512          | 2.29868    |


...

| 1997/2000     | ILSVRC2012_val_00020720.JPEG             | 1.79079    | 0.00000    | 512          | 2.36340    |


100%|██████████| 32/32 [00:02<00:00, 13.67it/s]


| 1998/2000     | ILSVRC2012_val_00008183.JPEG             | 74.30656   | 0.00000    | 512          | 2.36445    |


100%|██████████| 32/32 [00:02<00:00, 13.74it/s]

| 1999/2000     | ILSVRC2012_val_00031879.JPEG             | 0.00000    | 1.00000    | 512          | 2.35483    |
| Average    | -                                        | 41.57491   | 17.40000   | 512.0        | 2.36799    |


Insertion
- Grad CAM: 0.531940429590203
- Grad CAM++: 0.533122277181148
- Score CAM: 0.559063476664377
- Grad-Score CAM: 0.559786926339719

Deletion
- Grad CAM: 0.118457254563162
- Grad CAM++: 0.124108827578648
- Score CAM: 0.12806089364544
- Grad-Score CAM: 0.12670076260866

Avg Drop
- Score CAM: 41.57491
- Grad-Score CAM: 41.51647

Increase in confidence
- Score CAM: 17.4
- Grad-Score CAM: 17.3

Avg feature maps
- Score CAM: 512
- Grad-Score CAM: 466

Avg Time (T4)
- Score CAM: 2.36799
- Grad-Score CAM: 2.16583